In [1]:
import random

In [24]:
inputviable=0

while inputviable==0: 
    bloquept = int(input('Ingrese el largo del bloque de texto plano: '))
    tamaniollave = int(input('Ingrese el largo de la llave a utilizar: '))
    tamaniosubllave = int(input('Ingrese el largo de la sub llave a utilizar'))
    cantSbox = int(input('Ingrese cantidad de S-box a usar: '))
    if (bloquept>=tamaniosubllave) and (tamaniollave>tamaniosubllave) and (cantSbox>0) and (tamaniosubllave>(bloquept/2) and ((bloquept/2)%cantSbox==0) and (tamaniosubllave%cantSbox==0)):
        inputviable=1
    else:
        print('Los datos ingresados no tienen sentido para el algoritmo DES, ingrese nuevos datos')

mediobloquept = int(bloquept/2)
entradaSbox = int(tamaniosubllave/cantSbox)
salidaSbox = int(mediobloquept/cantSbox)

print('Configuración final')
print('Tamaño de bloque texto plano: ', bloquept,' bits')
print('Tamaño de llave: ', tamaniollave,' bits')
print('Tamaño de sub-llave: ', tamaniosubllave,' bits')
print('Cantidad de S-Cajas: ', cantSbox)
print('Bits de salida S-Cajas: ', salidaSbox,' bits')


Los datos ingresados no tienen sentido para el algoritmo DES, ingrese nuevos datos


ValueError: invalid literal for int() with base 10: ''

In [14]:

# Genera tabla de permutación 
def generatePermTab(n):
	table=[*range(1,n+1,1)]
	for i in range(len(table)-1):
		random_index = random.randint(0, (len(table)-1))
		temp = table.pop(random_index)
		table.append(temp)
	return table
	
# Genera tabla de expansion para el bloque que va a pasar por las s-box
def generateExpTab(mediobloquept, tamaniosubllave):
	values= [*range(1,mediobloquept+1, 1)]
	exp=[]
	for i in range(tamaniosubllave):
		exp.append(values[random.randint(0,(len(values)-1))])
	return exp

# Convierte texto plano a hex
def pt2hex(s):
	hex_s = s.encode('utf-8')
	hex_s = hex_s.hex()
	return hex_s.upper()

# Conversión de hex a decimal
def hex2bin(s):
	mp = {'0': "0000",
		'1': "0001",
		'2': "0010",
		'3': "0011",
		'4': "0100",
		'5': "0101",
		'6': "0110",
		'7': "0111",
		'8': "1000",
		'9': "1001",
		'A': "1010",
		'B': "1011",
		'C': "1100",
		'D': "1101",
		'E': "1110",
		'F': "1111"}
	bin = ""
	for i in range(len(s)):
		bin = bin + mp[s[i]]
	return bin

# Convierte binario a hex
def bin2hex(s):
	mp = {"0000": '0',
		"0001": '1',
		"0010": '2',
		"0011": '3',
		"0100": '4',
		"0101": '5',
		"0110": '6',
		"0111": '7',
		"1000": '8',
		"1001": '9',
		"1010": 'A',
		"1011": 'B',
		"1100": 'C',
		"1101": 'D',
		"1110": 'E',
		"1111": 'F'}
	hex = ""
	for i in range(0, len(s), 4):
		ch = ""
		ch = ch + s[i]
		ch = ch + s[i + 1]
		ch = ch + s[i + 2]
		ch = ch + s[i + 3]
		hex = hex + mp[ch]

	return hex

# Convierte binario a decimal
def bin2dec(binary):

	binary1 = binary
	decimal, i, n = 0, 0, 0
	while(binary != 0):
		dec = binary % 10
		decimal = decimal + dec * pow(2, i)
		binary = binary//10
		i += 1
	return decimal

# Convierte decimal a binario
def dec2bin(num):
	res = bin(num).replace("0b", "")
	if(len(res) % 4 != 0):
		div = len(res) / 4
		div = int(div)
		counter = (4 * (div + 1)) - len(res)
		for i in range(0, counter):
			res = '0' + res
	return res

# Realiza una permutación de bits segun alguna tabla de permutación o expansion, devuelve un string de n bits
def permute(k, arr, n):
	permutation = ""
	for i in range(0, n):
		permutation = permutation + k[arr[i] - 1]
	return permutation

# Realiza shift de n bits hacia la izquierda s
def shift_left(k, nth_shifts):
	s = ""
	for i in range(nth_shifts):
		for j in range(1, len(k)):
			s = s + k[j]
		s = s + k[0]
		k = s
		s = ""
	return k

# calcula xor entre dos binarios a y b
def xor(a, b):
	ans = ""
	for i in range(len(a)):
		if a[i] == b[i]:
			ans = ans + "0"
		else:
			ans = ans + "1"
	return ans

In [15]:
# Las s-cajas toman 6 bits y devuelven 4
def sbox(entrada, salida):
	return entrada[:len(entrada) - (len(entrada)-salida)]

In [16]:
# Genera la tabla de permutacion inicial
initial_perm = generatePermTab(bloquept)
print(initial_perm)

[8, 9, 1, 4, 6, 2, 7, 12, 3, 10, 11, 5]


In [17]:
# Genera tabla de expansion de bloque
exp_d = generateExpTab(mediobloquept,tamaniosubllave)
print(exp_d)

[6, 3, 5, 5, 6, 3, 4, 5]


In [18]:
# Genera tabla de permutacion directa
per = generatePermTab(mediobloquept)
print(per)

[1, 6, 5, 4, 2, 3]


In [19]:
# Genera tabla de permutacion final
final_perm = generatePermTab(bloquept)
print(final_perm)

[2, 5, 6, 10, 11, 1, 12, 8, 3, 4, 7, 9]


In [20]:
def encrypt(pt, rkb):
	print('Texto plano en hex: ', pt)
	pt = hex2bin(pt)
	print('Texto plano en binario:', pt)

	# Permutación inicial
	pt = permute(pt, initial_perm, bloquept)
	print("Despues de la primera permutación: ", pt)

	# El bloque entero se separa en 2 partes, izquierda y derecha
	left = pt[0:mediobloquept]
	right = pt[mediobloquept:bloquept]
	for i in range(0, 16):
		# Se expande el medio bloque al tamaño de la sub llave
		right_expanded = permute(right, exp_d, tamaniosubllave)

		# XOR entre RoundKey[i] y el bloque derecho expandido
		xor_x = xor(right_expanded, rkb[i])

		# El resultado del xor pasa por las s-cajas correspondientes
		sbox_str = ""
		for j in range(0, cantSbox):
			sixbits=xor_x[j*entradaSbox:(entradaSbox*6)+6]
			sbox_str = sbox_str + sbox(sixbits,salidaSbox)
		
		# Se realiza una permutacion adicional, la cual es directa
		sbox_str = permute(sbox_str, per, mediobloquept)

		# Se realiza un xor entre el medio bloque izquierdo y el resultado de las s-cajas
		result = xor(left, sbox_str)
		left = result

		# Se intercambia el lado izquierdo con el derecho
		if(i != 15):
			left, right = right, left
		print("Ronda ", i + 1, " Bloque izquierdo: ", left,
			" Bloque derecho: ", right, " Sub-llave: ", rkb[i])

	# Se concatena izquierda + derecha
	combine = left + right

	# Se realiza la permutacion final de bits
	cipher_text = permute(combine, final_perm, bloquept)
	return cipher_text

In [21]:
entrada = ""
llave = ""

while (len(entrada)*8)<bloquept:
    entrada=input('Ingrese texto plano de al menos el tamaño del bloque elegido: ')

while (len(llave)*8)<tamaniollave:
    llave=input('Ingrese llave de al menos el tamaño elegido: ')

pt = pt2hex(entrada)
key = pt2hex(llave)

print('Texto plano: ', entrada)
print('Llave: ', llave)
print('Entrada en hexadecimal: ', pt)
print('Llave en hexadecimal: ', key)


Texto plano:  AB
Llave:  DA
Entrada en hexadecimal:  4142
Llave en hexadecimal:  4441


In [22]:
# Generación de sub-llaves
# --llave de hexa a binario
key = hex2bin(key)

# Orden de shifts para generar las 16 sub-llaves distintas
shift_table = [1, 1, 2, 2,
			2, 2, 2, 2,
			1, 2, 2, 2,
			2, 2, 2, 1]

# Se genera tabla para comprimir las llaves
key_comp = generatePermTab(tamaniosubllave)

# Llave se divide en 2 partes
left = key[0:int(tamaniollave/2)] 
right = key[int(tamaniollave/2):tamaniollave] 

rkb = [] # rkb para sub-llaves en binario


# Se generan 16 sub llaves
for i in range(0, 16):
	# Se realizan los shifts segun la tabla de arriba
	left = shift_left(left, shift_table[i])
	right = shift_left(right, shift_table[i])

	# Se combinan ambas partes
	combine_str = left + right

	# Se comprime la llave al tamaño de las sub-llaves elegido
	round_key = permute(combine_str, key_comp, tamaniosubllave)
	#Lista con 16 sub-llaves
	rkb.append(round_key)

In [23]:
print("Cifrado")
cipher_text = bin2hex(encrypt(pt, rkb))
print("Texto cifrado en hex: ", cipher_text)

Cifrado
Texto plano en hex:  4142
Texto plano en binario: 0100000101000010
Despues de la primera permutación:  100001000100
Ronda  1  Bloque izquierdo:  000100  Bloque derecho:  111001  Sub-llave:  00010100
Ronda  2  Bloque izquierdo:  111001  Bloque derecho:  111111  Sub-llave:  00101010
Ronda  3  Bloque izquierdo:  111111  Bloque derecho:  000010  Sub-llave:  00001001
Ronda  4  Bloque izquierdo:  000010  Bloque derecho:  101110  Sub-llave:  00000011
Ronda  5  Bloque izquierdo:  101110  Bloque derecho:  001100  Sub-llave:  00101010
Ronda  6  Bloque izquierdo:  001100  Bloque derecho:  110000  Sub-llave:  00001001
Ronda  7  Bloque izquierdo:  110000  Bloque derecho:  011100  Sub-llave:  00000011
Ronda  8  Bloque izquierdo:  011100  Bloque derecho:  111111  Sub-llave:  00101010
Ronda  9  Bloque izquierdo:  111111  Bloque derecho:  001001  Sub-llave:  11000100
Ronda  10  Bloque izquierdo:  001001  Bloque derecho:  110001  Sub-llave:  10010000
Ronda  11  Bloque izquierdo:  110001  Bloque 